In [ ]:
#segmentation of two eye images
import cv2
import numpy as np
from scipy.signal import find_peaks

# iris segmentation
def iris_segmentation(img):
    #img to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Gaussian blur (noise removal)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # adaptive thresholding (binarize img)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # morphological closing (fill  gaps)
    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # finding vertical iris boundaries
    dy = np.diff(closing, axis=0)
    peaks, _ = find_peaks(np.sum(dy, axis=1), distance=20)

    # extracting iris region
    iris = img[peaks[0]:peaks[-1], :]

    return iris
#loading images
img1 = cv2.imread('/content/1.jpg')
img2 = cv2.imread('/content/2.jpg')

# iris segmentation  performance on each image
iris1 = iris_segmentation(img1)
iris2 = iris_segmentation(img2)

# saving
cv2.imwrite('iris1.jpg', iris1)
cv2.imwrite('iris2.jpg', iris2)

True

In [ ]:
# normalisation for iris1
import cv2
import numpy as np

# Loading iris image1
img = cv2.imread('/content/iris1.jpg')

# grayscale conversion
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Gaussian blur to reduce noise
gray = cv2.GaussianBlur(gray, (5,5), 0)

# Applying  threshold
ret, thresh = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)

# Finding contours of the iris
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Selecting largest contour,
iris_contour = max(contours, key=cv2.contourArea)

# Finding the center and radius of the iris
(x, y), radius = cv2.minEnclosingCircle(iris_contour)
center = (int(x), int(y))
radius = int(radius)

# visualization
cv2.circle(img, center, radius, (0, 255, 0), 2)

# normalization by cropping and resizing the iris region
iris = img[int(y)-radius:int(y)+radius, int(x)-radius:int(x)+radius]
iris = cv2.resize(iris, (200, 100))

# Save the normalized iris image
cv2.imwrite('normalized_iris1.jpg', iris)

True

In [ ]:
# normalisation for iris2
import cv2
import numpy as np

# Loading
img = cv2.imread('/content/iris2.jpg')

# Conversion
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur (reduce noise)
gray = cv2.GaussianBlur(gray, (5,5), 0)

# Applying threshold to segment
ret, thresh = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)

# Finding  contours of the iris
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Selecing  largest contour, which should be the iris
iris_contour = max(contours, key=cv2.contourArea)

# Finding center and radius of the iris
(x, y), radius = cv2.minEnclosingCircle(iris_contour)
center = (int(x), int(y))
radius = int(radius)

# visualization
cv2.circle(img, center, radius, (0, 255, 0), 2)

# Perform normalization
iris = img[int(y)-radius:int(y)+radius, int(x)-radius:int(x)+radius]
iris = cv2.resize(iris, (200, 100))

# Saving
cv2.imwrite('normalized_iris2.jpg', iris)

True

In [ ]:
# feature encoding for normalized iris  image 1

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

# Loading pre-trained VGG16 model (without the last layer)
model = VGG16(weights='imagenet', include_top=False)

# Load normalized image from file
img_path = '/content/normalized_iris1.jpg'
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Encode image features
features = model.predict(img_array)

# Saving
np.save('features1.npy', features)


1/1 [==============================] - 1s 969ms/step


In [ ]:
# feature encoding for normlised  iris image 2

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16

# Load pre-trained VGG16 model (without the last layer)
model = VGG16(weights='imagenet', include_top=False)

# Loading normalized image
img_path = '/content/normalized_iris2.jpg'
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Encode image features
features = model.predict(img_array)

# Saving
np.save('features2.npy', features)


1/1 [==============================] - 1s 709ms/step


In [ ]:
#Matching
import numpy as np

# loading the encoded features for the two iris images
features1 = np.load('/content/features1.npy')
features2 = np.load('/content/features2.npy')

# calculating the Hamming Distance between the two feature vectors
distance = np.count_nonzero(features1 != features2)

# set a threshold value for similarity
threshold = 50

# Matching
if distance <= threshold:
    print("The two iris images match!")
else:
    print("The two iris images do not match.")


The two iris images do not match.
